#Silver Layer - Cleaned and Transformed Data

The Silver layer focuses on preparing raw Bronze data for analytical use by:
- Cleaning invalid or inconsistent values
- Standardizing codes and formats
- Removing duplicates and bad records
- Correcting data types and enforcing basic quality rules

No business aggregations or KPI logic are applied at this stage.
Those transformations are intentionally deferred to the Gold layer. Also, no data models are created in the silver layer.

Each table is processed independently to ensure traceability, auditability, and ease of validation.

Post-load data quality validation is performed using dedicated SQL test scripts stored in the /tests folder of the repository. These checks validate data consistency, standardization, and structural integrity before data is promoted to the Gold layer.


##Create Silver Tables

In [0]:
%sql
USE CATALOG workspace;
USE SCHEMA silver;

-- CRM tables

DROP TABLE IF EXISTS silver.crm_cust_info;
CREATE TABLE workspace.silver.crm_cust_info (
    cst_id             INT,
    cst_key            STRING,
    cst_firstname      STRING,
    cst_lastname       STRING,
    cst_marital_status STRING,
    cst_gndr           STRING,
    cst_create_date    DATE
)
USING DELTA;

DROP TABLE IF EXISTS silver.crm_prd_info;
CREATE TABLE workspace.silver.crm_prd_info (
    prd_id            INT,
    cat_id            STRING,
    prd_key           STRING,
    prd_nm            STRING,
    prd_cost          DOUBLE,
    prd_line          STRING,
    prd_start_dt      DATE,
    prd_end_dt        DATE
)
USING DELTA;

DROP TABLE IF EXISTS silver.crm_sales_details;
CREATE TABLE workspace.silver.crm_sales_details (
    sls_ord_num       STRING,
    sls_prd_key       STRING,
    sls_cust_id       INT,
    sls_order_dt      DATE,
    sls_ship_dt       DATE,
    sls_due_dt        DATE,
    sls_sales         INT,
    sls_quantity      INT,
    sls_price         INT
)
USING DELTA;

-- ERP tables

DROP TABLE IF EXISTS silver.erp_cust_az12;
CREATE TABLE workspace.silver.erp_cust_az12 (
    cid STRING,
    bdate DATE,
    gen STRING
)
USING DELTA;

DROP TABLE IF EXISTS silver.erp_loc_a101;
CREATE TABLE workspace.silver.erp_loc_a101 (
    cid STRING,
    cntry STRING
)
USING DELTA;

DROP TABLE IF EXISTS silver.erp_px_cat_g1v2;
CREATE TABLE workspace.silver.erp_px_cat_g1v2 (
    id STRING,
    cat STRING,
    subcat STRING,
    maintenance STRING
)
USING DELTA;



##Load & Transform Data

In [0]:
%sql
-- Loading silver.crm_cust_info
INSERT OVERWRITE workspace.silver.crm_cust_info
SELECT
    cst_id,
    cst_key,
    TRIM(cst_firstname) AS cst_firstname,
    TRIM(cst_lastname) AS cst_lastname,
    CASE
        WHEN UPPER(TRIM(cst_marital_status)) = 'S' THEN 'Single'
        WHEN UPPER(TRIM(cst_marital_status)) = 'M' THEN 'Married'
        ELSE 'n/a'
    END AS cst_marital_status,  -- Normalized marital status values to readable format
    CASE
        WHEN UPPER(TRIM(cst_gndr)) = 'F' THEN 'Female'
        WHEN UPPER(TRIM(cst_gndr)) = 'M' THEN 'Male'
        ELSE 'n/a'
    END AS cst_gndr,  -- Normalized gender values to readable format
    cst_create_date
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY cst_id ORDER BY cst_create_date DESC) AS rn
    FROM workspace.bronze.crm_cust_info
    WHERE cst_id IS NOT NULL
) t
WHERE rn = 1;  -- Select the most recent record per customer

-- Loading silver.crm_prd_info
INSERT OVERWRITE workspace.silver.crm_prd_info
SELECT
    prd_id,
    REPLACE(SUBSTRING(prd_key, 1, 5), '-', '_') AS cat_id,  -- Extract category ID
    SUBSTRING(prd_key, 7) AS prd_key,                       -- Extract product key
    prd_nm,
    COALESCE(prd_cost, 0) AS prd_cost,
    CASE
        WHEN UPPER(TRIM(prd_line)) = 'M' THEN 'Mountain'
        WHEN UPPER(TRIM(prd_line)) = 'R' THEN 'Road'
        WHEN UPPER(TRIM(prd_line)) = 'S' THEN 'Other Sales'
        WHEN UPPER(TRIM(prd_line)) = 'T' THEN 'Touring'
        ELSE 'n/a'
    END AS prd_line,  -- Map product line codes to descriptive values
    CAST(prd_start_dt AS DATE) AS prd_start_dt,
    CAST(
        LEAD(prd_start_dt) OVER (PARTITION BY prd_key ORDER BY prd_start_dt) - INTERVAL 1 DAY
        AS DATE
    ) AS prd_end_dt  -- Calculate end date as one day before the next start date
FROM workspace.bronze.crm_prd_info;

-- Loading crm_sales_details
INSERT OVERWRITE workspace.silver.crm_sales_details
SELECT
    sls_ord_num,
    sls_prd_key,
    sls_cust_id,
    CASE WHEN sls_order_dt = 0 OR LENGTH(sls_order_dt) != 8 THEN NULL
         ELSE TO_DATE(CAST(sls_order_dt AS STRING), 'yyyyMMdd') END AS sls_order_dt,
    CASE WHEN sls_ship_dt = 0 OR LENGTH(sls_ship_dt) != 8 THEN NULL
         ELSE TO_DATE(CAST(sls_ship_dt AS STRING), 'yyyyMMdd') END AS sls_ship_dt,
    CASE WHEN sls_due_dt = 0 OR LENGTH(sls_due_dt) != 8 THEN NULL
         ELSE TO_DATE(CAST(sls_due_dt AS STRING), 'yyyyMMdd') END AS sls_due_dt,
    CASE
        WHEN sls_sales IS NULL OR sls_sales <= 0
             OR sls_sales != sls_quantity * ABS(sls_price)
        THEN sls_quantity * ABS(sls_price)
        ELSE sls_sales
    END AS sls_sales,  -- Recalculate sales if original value is missing or incorrect
    sls_quantity,
    CASE
        WHEN sls_price IS NULL OR sls_price <= 0
        THEN sls_sales / NULLIF(sls_quantity, 0)
        ELSE sls_price  -- Derive price if original value is invalid
    END AS sls_price
FROM workspace.bronze.crm_sales_details;

-- Loading erp_cust_az12
INSERT OVERWRITE workspace.silver.erp_cust_az12 
SELECT
	CASE
		WHEN cid LIKE 'NAS%' THEN SUBSTRING(cid, 4) -- Remove 'NAS' prefix if present
		ELSE cid
	END AS cid, 
	CASE
		WHEN bdate > CURRENT_DATE() THEN NULL
		ELSE bdate
	END AS bdate, -- Set future birthdates to NULL
	CASE
		WHEN UPPER(TRIM(gen)) IN ('F', 'FEMALE') THEN 'Female'
		WHEN UPPER(TRIM(gen)) IN ('M', 'MALE') THEN 'Male'
		ELSE 'n/a'
	END AS gen -- Normalize gender values and handle unknown cases
FROM workspace.bronze.erp_cust_az12;

-- Loading erp_loc_a101
INSERT OVERWRITE workspace.silver.erp_loc_a101
SELECT
    REPLACE(cid, '-', '') AS cid,
    CASE
        WHEN TRIM(cntry) = 'DE' THEN 'Germany'
        WHEN TRIM(cntry) IN ('US', 'USA') THEN 'United States'
        WHEN cntry IS NULL OR TRIM(cntry) = '' THEN 'n/a'
        ELSE TRIM(cntry)
    END AS cntry  -- Normalize and Handle missing or blank country codes
FROM workspace.bronze.erp_loc_a101;

-- Loading erp_px_cat_g1v2
INSERT OVERWRITE workspace.silver.erp_px_cat_g1v2
SELECT
    id,
    cat,
    subcat,
    maintenance
FROM workspace.bronze.erp_px_cat_g1v2;